<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(11_2_RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98_%EC%8A%A4%ED%8C%B8_%EB%A9%94%EC%9D%BC_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 스팸 메일 분류하기
- https://wikidocs.net/22894
---
캐글(https://www.kaggle.com/uciml/sms-spam-collection-dataset)에서 제공하는 정상메일과 스팸메일이 섞여 있는 스팸 메일 데이터를 가지고, 데이터 전처리 후 바닐라RNN을 이용한 스팸 메일 분류기 구현

### 스팸 메일 데이터 이해

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
data = pd.read_csv((io.StringIO(uploaded['spam.csv'].decode('latin1'))))

In [ ]:
print(f"총 샘플의 수 : {len(data)}")

In [ ]:
data.head()

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().values.any()

In [ ]:
data.v1.nunique(),data.v2.nunique()

In [ ]:
data = data.drop_duplicates(subset = ['v2']).copy()

In [ ]:
print(f"총 샘플의 수 : {len(data)}")

In [ ]:
data.v1.value_counts().plot.bar()

In [ ]:
data.groupby('v1').size().reset_index(name = "count")

In [ ]:
X_data = data.v2.copy()
y_data = data.v1.copy()
print(f"메일 본문의 개수 : {len(X_data)}")
print(f"메일 레이블의 개수 : {len(y_data)}")

In [ ]:
#정수인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

In [ ]:
print(sequences[:5])

In [ ]:
word_to_index = tokenizer.word_index
print(word_to_index)

In [ ]:
# 빈도수가 1개인 단어의 비율
threshold = 2
total_cnt = len(word_to_index) #단어의 수
rare_cnt = 0 #threshold보다 작은 빈도수를 가진 단어의 개수 카운트
total_freq = 0 #훈련 데이터의 전체 단어 빈도수 합
rare_freq = 0 #theshold보다 작은 단어의 등장 빈도수 합

for key,value in tokenizer.word_counts.items():
  total_freq += value

  #단어의 등장 빈도수가 threshold보다 작으면
  if value < threshold:
    rare_cnt += 1
    rare_freq += value

print('등장 빈도수가 %s 미만인 희귀 단어의 수 %s',(threshold, rare_cnt))
print('단어 집합(vocab)에서 희귀 단어의 비율: ',round((rare_cnt/total_cnt)*100,2))
print("전체 등장 빈도에서 희귀 단어 등장 비율: ", round(rare_freq/total_freq*100,2))

만약 등장 빈도가 1회 이하인 단어를 제외하고자 할 경우
- tokenizer = Tokenizer(num_words = (total_cnt = rare_cnt + 1), 단 이 과정에선 제외
---

padding
-padding 시 padding을 위한 토큰 0번 단어를 고려하여 단어 집합의 크기 vocab_size는 +1


In [ ]:
vocab_size = len(word_to_index) + 1
print("단어 집합의 크기 : ",vocab_size)

In [ ]:
#테스트 및 훈련셋 나누기 8:2
n_of_train = int(len(sequences) * 0.8)
n_of_test = int(len(sequences) - n_of_train)

print(f"훈련 데이터셋 / 테스트 데이터셋 개수 : {n_of_train} / {n_of_test}")

In [ ]:
X_data = sequences
print('메일의 최대 길이 : %d' % max(len(l) for l in X_data))
print('메일의 평균 길이 : %.1f' % (sum(map(len,X_data))/len(X_data)))

plt.hist([len(s) for s in X_data], bins = 50)
plt.xlabel('length of samples')
plt.ylabel('num of samples')
plt.show()

In [ ]:
max_len = max(len(l) for l in X_data) # 가장 긴 메일의 길이가 189

data = pad_sequences(X_data, maxlen = max_len,padding = 'post')
print("훈련 데이터의 크기(shape) : ", data.shape)

In [ ]:
X_train = data[:n_of_train]
y_train = np.array(y_data[:n_of_train])

X_test = data[-n_of_test:]
y_test = np.array(y_data[-n_of_test:])

## RNN으로 스팸 메일 분류

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim=32)) #임베딩 벡터 차원 32
model.add(SimpleRNN(units = 32)) #메모리셀의 hidden_size = 32, 즉 메모리 셀의 output = 32
model.add(Dense(units = 1, activation = 'sigmoid')) #최종 output 1개(이진분류), 활성화함수 : 시그모이드

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(x = X_train, y = y_train, epochs = 4, batch_size = 64,validation_split=0.2)

In [ ]:
epochs = range(1,len(history.history['acc'])+1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'],loc = 'upper right')
plt.show()